In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

In [38]:
from pycaret.regression import *
# 데이터 분리 (종속변수 지정)
exp1 = setup(data=train, target='임신 성공 여부', normalize = True, session_id=777)

# 여러 모델 비교 및 성능 확인
# best_model = compare_models()

ValueError: Key backend: 'inline' is not a valid value for backend; supported values are ['GTK3Agg', 'GTK3Cairo', 'GTK4Agg', 'GTK4Cairo', 'MacOSX', 'nbAgg', 'QtAgg', 'QtCairo', 'Qt5Agg', 'Qt5Cairo', 'TkAgg', 'TkCairo', 'WebAgg', 'WX', 'WXAgg', 'WXCairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']

In [45]:
model = ExtraTreesClassifier(random_state=42)

model.fit(X_train_encoded, y)

ExtraTreesClassifier(random_state=42)

In [46]:
pred_proba = model.predict_proba(X_test_encoded)[:, 1]

In [47]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_proba

In [48]:
sample_submission.to_csv('./submit_1.csv', index=False)